In [1]:
import pandas as pd
import numpy as np

URL = 'https://venturebeat.com/%s-%s-npd'
months = ['July', 'August', 'October', 'November', 'December']
year = '2020'

combine_df = pd.DataFrame(columns = ['Date', 'Total Video Game Sales', 'Video Game Hardware',
                                     'Video Game Content','Video Game Accessories'])
i = 0

for m in months:
    df = pd.read_html(URL % (m, year))
    if m in ['July', 'October']:
        df[i] = df[i].T
    elif m =='August':
        df[i] = df[i].T
        df[i] = df[i].drop(df[i].columns[[1,6]], axis=1)
    else:
        df[i] = df[i].drop(df[i].columns[1], axis=1).T
        df[i] = df[i].drop(df[i].columns[[1,6]], axis=1)
    df[i] = df[i].rename(columns=df[i].iloc[0]).iloc[1:-1]
    df[i] = df[i].rename(columns={df[i].columns[0] : 'Date', df[i].columns[3] : 'Video Game Content'})
    combine_df = pd.concat([combine_df, df[i]], ignore_index=True)
combine_df

,Date,Total Video Game Sales,Video Game Hardware,Video Game Content,Video Game Accessories
0,Jul’19,"$2,723",$170,"$2,426",$127
1,Jul’20,"$3,587",$166,"$3,251",$170
2,Aug’19,"$2,431",$167,"$2,147",$117
3,Aug’20,"$3,330",$229,"$2,935",$166
4,Oct’19,"$3,358",$183,"$3,054",$121
5,Oct’20,"$3,836",$259,"$3,429",$148
6,Nov 2019,"$5,162",$897,"$3,974",$291
7,Nov 2020,"$6,973","$1,415","$5,244",$314
8,Dec 2019,"$6,184",$978,"$4,732",$474
9,Dec 2020,"$7,702","$1,350","$5,806",$546


In [2]:
months = ['January','February','March']
year = '2021'

for m in months:
    if m == 'March':
        i = 1
    df = pd.read_html(URL % (m, year))
    df[i] = df[i].drop(df[i].columns[1], axis=1).T
    df[i] = df[i].drop(df[i].columns[[1,6]], axis=1)
    df[i] = df[i].rename(columns=df[i].iloc[0]).iloc[1:-1]
    df[i] = df[i].rename(columns={df[i].columns[0] : 'Date', df[i].columns[3] : 'Video Game Content'})
    combine_df = pd.concat([combine_df, df[i]], ignore_index=True)
combine_df

,Date,Total Video Game Sales,Video Game Hardware,Video Game Content,Video Game Accessories
0,Jul’19,"$2,723",$170,"$2,426",$127
1,Jul’20,"$3,587",$166,"$3,251",$170
2,Aug’19,"$2,431",$167,"$2,147",$117
3,Aug’20,"$3,330",$229,"$2,935",$166
4,Oct’19,"$3,358",$183,"$3,054",$121
5,Oct’20,"$3,836",$259,"$3,429",$148
6,Nov 2019,"$5,162",$897,"$3,974",$291
7,Nov 2020,"$6,973","$1,415","$5,244",$314
8,Dec 2019,"$6,184",$978,"$4,732",$474
9,Dec 2020,"$7,702","$1,350","$5,806",$546


In [3]:
months = ['January','February','March', 'April', 'June']
year = '2020'

df_before_july = pd.DataFrame(columns = ['Date', 'Total Video Game Sales', 'Video Games Hardware',
                                         'PC & Video Games Software','Video Game Accessories & Game Cards'])
for m in months:
    df = pd.read_html(URL % (m, year))
    
    df[0] = df[0].drop(df[0].columns[1], axis=1).T
    if m in ['March', 'April', 'June']:
        df[0] = df[0].drop(df[0].columns[1], axis=1)
    df[0] = df[0].rename(columns=df[0].iloc[0]).iloc[1:-1]
    df[0] = df[0].rename(columns={df[0].columns[0]: 'Date', df[0].columns[1]: 'Total Video Game Sales', 
                                  df[0].columns[2]: 'Video Games Hardware', df[0].columns[3]: 'PC & Video Games Software',
                                  df[0].columns[4]: 'Video Game Accessories & Game Cards'})
    df_before_july = pd.concat([df_before_july, df[0]], ignore_index=True)
df_before_july

,Date,Total Video Game Sales,Video Games Hardware,PC & Video Games Software,Video Game Accessories & Game Cards
0,Jan’19,$918m,$199m,$451m,$268m
1,Jan’20,$678m,$129m,$311m,$238m
2,Feb’19,"$1,061m",$277m,$477m,$308m
3,Feb’20,$755m,$183m,$307m,$265m
4,Mar’19,"$1,186m",$282m,$550m,$354m
5,Mar’20,"$1,597m",$461m,$739m,$397m
6,Apr’19,$846,$160,$428,$258
7,Apr’20,"$1,466",$420,$662,$384
8,Jun’20,"$1,178",$191,$570,$417


In [4]:
combine_df.to_csv('./Datasets/gamesales(July_2020_March_2021).csv')
df_before_july.to_csv('./Datasets/gamesales(Jan_2020_June_2020).csv')